## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [3]:
#Se importan las librerias a utilizar

import pandas as pd

/home/sebastian/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


----

##### Lectura de los datasets

In [5]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))


df_integrado = pd.read_csv("../../data/processed/datos_integrados_mc.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,0.0,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,0.0,True,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0,ok


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [8]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['completitud_fila','situacion_vivienda_ok']
col_eliminar_tarjetas = []

# Se ejecuta la operación
df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [9]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


Limpieza de los datos ( filtrado a nivel de filas)

In [14]:
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)  
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,ok
1,768805383.0,21,1000,2,5.0,PROPIA,9600,False,0.10,11.14,0,N,ok
2,818770008.0,25,5500,3,1.0,HIPOTECA,9600,False,0.57,12.87,1,N,err
3,713982108.0,23,35000,2,4.0,ALQUILER,65500,False,0.53,15.23,1,N,ok
4,710821833.0,24,35000,4,8.0,ALQUILER,54400,False,0.55,14.27,1,Y,ok


In [17]:
# Se filtran las filas con algún error detectado
print(f"filas antes del filtro:{df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]
temp_c = temp[temp['antiguedad_empleado']<=50]
temp_c2 = temp[temp['regla_pct_ingresos']== 'ok']


print(f"filas despues del filtro:{temp_c2.shape[0]}")

# Otro filtro posible:
# df_creditos['antiguedad_empleado'] > 50

# También por las reglas del negocio
# df[df.regla_pct_ingresos == 'err']


filas antes del filtro:10127
filas despues del filtro:10108


Integración de datos

In [19]:
df_integrado = pd.merge(df_creditos, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"filas despues del filtro:{temp_c2.shape[0]}")

filas despues del filtro:10108


Transformación de atributos

In [21]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')
df_final = pd.concat([estado_civil_N, df_integrado], axis=1)
df_final.head()



# Columna: estado_CREDITO
cambios_estado_credito = {
    0 : 'C',
    1:'P'
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')
df_final = pd.concat([estado_credito_N, df_integrado], axis=1)
df_final.head()
# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_credito_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,P,713061558.0,22,35000,3,123.0,ALQUILER,59000,False,0.59,16.02,1,Y,ok,36.0,CASADO,ACTIVO,1088.0,M,4010.0,UNIVERSITARIO_COMPLETO,Blue,24.0,2.0
1,C,768805383.0,21,1000,2,5.0,PROPIA,9600,False,0.10,11.14,0,N,ok,39.0,CASADO,ACTIVO,1144.0,M,12691.0,SECUNDARIO_COMPLETO,Blue,42.0,3.0
2,P,818770008.0,25,5500,3,1.0,HIPOTECA,9600,False,0.57,12.87,1,N,err,44.0,SOLTERO,ACTIVO,1291.0,F,8256.0,UNIVERSITARIO_COMPLETO,Blue,33.0,5.0
3,P,713982108.0,23,35000,2,4.0,ALQUILER,65500,False,0.53,15.23,1,N,ok,36.0,CASADO,ACTIVO,1887.0,M,3418.0,UNIVERSITARIO_COMPLETO,Blue,20.0,3.0
4,P,710821833.0,24,35000,4,8.0,ALQUILER,54400,False,0.55,14.27,1,Y,ok,54.0,CASADO,ACTIVO,1314.0,M,9095.0,DESCONOCIDO,Blue,26.0,1.0


Exportación de metadatos